In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/welding-defect-object-detection/The Welding Defect Dataset - v2/The Welding Defect Dataset - v2/data.yaml
/kaggle/input/welding-defect-object-detection/The Welding Defect Dataset - v2/The Welding Defect Dataset - v2/valid/labels/Not-perfect-welding-images_10_jpeg_jpg.rf.938705132c676bde86605b449fbfd55e.txt
/kaggle/input/welding-defect-object-detection/The Welding Defect Dataset - v2/The Welding Defect Dataset - v2/valid/labels/Screenshot-2022-12-05-223639_jpeg.rf.86f504360592eab360a0ad9923d695ab.txt
/kaggle/input/welding-defect-object-detection/The Welding Defect Dataset - v2/The Welding Defect Dataset - v2/valid/labels/crack-welding-images_21_jpeg_jpg.rf.a8fe58377f24d954a14e75c731321a8b.txt
/kaggle/input/welding-defect-object-detection/The Welding Defect Dataset - v2/The Welding Defect Dataset - v2/valid/labels/d18b341f-e637a315-c21_jpg.rf.12173f7ad5047c3c89e3bee036329565.txt
/kaggle/input/welding-defect-object-detection/The Welding Defect Dataset - v2/The Welding Defect

# IMPORTING LIBRARIES

In [2]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import plotly.express as px
import shutil
import random

from PIL import Image
from IPython.display import display


In [3]:
train_path="/kaggle/input/welding-defect-object-detection/The Welding Defect Dataset/The Welding Defect Dataset/train"
test_path="/kaggle/input/welding-defect-object-detection/The Welding Defect Dataset/The Welding Defect Dataset/test"
val_path="/kaggle/input/welding-defect-object-detection/The Welding Defect Dataset/The Welding Defect Dataset/valid"

# ANALYSIS OF DATASET

In [4]:
def getNumberOfImages(path):
    return f"Number Of Images in {path.split('/')[-1]}:{len(os.listdir(path+'/images'))}"

In [5]:
print(getNumberOfImages(train_path))
print(getNumberOfImages(test_path))
print(getNumberOfImages(val_path))



Number Of Images in train:839
Number Of Images in test:74
Number Of Images in valid:176


In [6]:
def getRandomImages(path,limit):
    for i in range(limit):
        img=path+"/images/"+np.random.choice(os.listdir(path+"/images"))
        img = Image.open(img)
        plt.figure(figsize=(8, 6)) 
        plt.imshow(img)
        plt.axis('off')  
        plt.title(f"{i+1} Image")
        plt.show()

        

In [7]:
getRandomImages(train_path,2)

NameError: name 'np' is not defined

In [ ]:
def getDistributionOfClasses(path):
    freq={0:0,1:0,2:0}
    for label in os.listdir(path+"/labels"):
        with open(path+"/labels/"+label) as lb:
            read=lb.read()
            read=read.strip()
            lines=read.split('\n')
            for line in lines:
                if line==" ":continue
                
                freq[int(line.split(" ")[0])]=freq[int(line.split(" ")[0])]+1
                
    return freq
            
            
            

In [ ]:
dist=getDistributionOfClasses(train_path)
classes={0:'Bad Weld', 1:'Good Weld', 2:'Defect'}
dist

In [ ]:
# plotting pie chart
x=list(dist.values())
labels = list(map(lambda x:classes[x],list(dist.keys())))



fig = px.pie(values=x, names=labels, title='Frequency OF Each Class In Train',
             color_discrete_sequence=px.colors.sequential.Viridis)

fig.show()


In [ ]:
# plotting bar graph 
x = list(dist.values())
labels = list(map(lambda x: classes[x], list(dist.keys())))

fig = px.bar(x=labels, y=x, title='Frequency OF Each Class In Train',
             color=labels, color_discrete_sequence=px.colors.sequential.Viridis)


fig.update_layout(xaxis_title='Classes', yaxis_title='Frequency')

fig.show()

# INSTALLING AND IMPORTING LIBRARIES FOR YOLO

In [ ]:
!pip install ultralytics
!pip install gradio

In [ ]:
from ultralytics import YOLO
from ultralytics.engine.results import Results
from IPython.display import display, Image
model =YOLO('yolov9m.pt')


# TRAINING OF THE MODEL

In [ ]:
results = model.train(data='/kaggle/input/welding-defect-object-detection/The Welding Defect Dataset/The Welding Defect Dataset/data.yaml',epochs=50,imgsz=640)

In [ ]:
model.val(data="/kaggle/input/welding-defect-object-detection/The Welding Defect Dataset/The Welding Defect Dataset/data.yaml")

# SAVE THE MODEL

In [ ]:
model_path = '/kaggle/working/yolov8n.pt'
model.save(model_path)

In [ ]:
import pandas as pd
pd.read_csv("/kaggle/working/runs/detect/train/results.csv")

In [ ]:
image = Image.open("/kaggle/working/runs/detect/train/confusion_matrix_normalized.png")
plt.figure(figsize=(20, 10))  

plt.imshow(image)
plt.axis('off')  
plt.show()

# PREDICTION ON RANDOM IMAGES OF TRAIN DATA

In [ ]:

def predictRandomImages(model,path,save_path,batch=5):
    images=os.listdir(path+"/images")
    random_images=random.sample(images,batch)
    for image in random_images:
        prediction=model(path+"/images/"+image)
        for pred in prediction:
            pred.plot(save=True,filename=save_path+image)
            display(Image.open(save_path+image))
        
    
    
    

In [ ]:
welding_model=YOLO(model_path)

In [ ]:
shutil.rmtree("prediction")
os.makedirs("prediction")
predictRandomImages(welding_model,test_path,"/kaggle/working/prediction/")

# DOWNLOADING FOLDER FROM KAGGLE NOTEBOOK

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))


In [ ]:
download_file("/kaggle/working/prediction","yolo_predictions")

# THE END